In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge,Lasso,ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV,LassoCV,ElasticNetCV
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [ ]:
df=pd.read_csv("/kaggle/input/used-car-dataset-ford-and-mercedes/vauxhall.csv")

In [ ]:
df.head()

In [ ]:
df["transmission"].unique()

In [ ]:
df["transmission"].value_counts()

In [ ]:
filt=(df["transmission"]=="Other")
df.loc[filt]

In [ ]:
group=df.groupby("model")

In [ ]:
group["engineSize"].value_counts(normalize=False).loc[[" Mokka"]]

In [ ]:
filt1=((df["model"]==" Mokka") & (df["engineSize"]==1.4) & (df["fuelType"]=="Petrol") & (df["year"]==2019))
df.loc[filt1]

In [ ]:
df.loc[7319]["transmission"]= "Manual"

In [ ]:
df.loc[7319,"transmission"]="Manual"

In [ ]:
df.loc[7319]["transmission"]

In [ ]:
df.drop(index=[6704,7422],inplace=True)

In [ ]:
df["fuelType"].value_counts()

In [ ]:
filt=(df["fuelType"]=="Hybrid")
df.loc[filt]

In [ ]:
df["model"].value_counts()

In [ ]:
df.drop(index=[13315,13316],inplace=True)

In [ ]:
df["model"].value_counts()

In [ ]:
filt=(df["model"]==" Ampera")
df.loc[filt]

In [ ]:
df.drop(index=13317,inplace=True)

In [ ]:
df["tax"].nunique()

In [ ]:
import missingno as msno
import seaborn as sns

In [ ]:
df.corr()

In [ ]:
sns.boxplot(data=df,x="tax");

In [ ]:
filt=(df["engineSize"]>2.5) 
df.loc[filt]

In [ ]:
df.drop(df.loc[filt].index,inplace=True)

In [ ]:
sns.boxplot(data=df,x="tax");

In [ ]:
filt=(df["tax"]>400) 
df.loc[filt]

In [ ]:
sns.boxplot(data=df,x="mpg");

In [ ]:
filt=(df["mpg"]<33) 
df.loc[filt]

In [ ]:
df.drop(df.loc[filt].index,inplace=True)

In [ ]:
sns.boxplot(data=df,x="tax");

In [ ]:
filt=((df["tax"]<20) & (df["mpg"]<70) )
df.loc[filt].count()

In [ ]:
sns.boxplot(data=df,x="year");

In [ ]:
df["year"].value_counts()

In [ ]:
filt=(df["year"]<2010)
df.drop(df.loc[filt].index,inplace=True)

In [ ]:
sns.boxplot(data=df,x="year");

In [ ]:
sns.boxplot(data=df,x="mileage");

In [ ]:
filt=(df["mileage"]>140000)
df.drop(df.loc[filt].index,inplace=True)

In [ ]:
sns.boxplot(data=df,x="mileage");

In [ ]:
df["model"].value_counts()

In [ ]:
filt=(df["model"]==" Kadjar") | (df["model"]==" Cascada") | (df["model"]==" Agila") | (df["model"]==" Vivaro") | ((df["model"]==" Antara")| (df["model"]==" Zafira Tourer") | (df["model"]=="GTC"))
df.drop(df.loc[filt].index,inplace=True)

In [ ]:
df["model"].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbe=LabelEncoder()
df["conv_fuelType"]=lbe.fit_transform(df["fuelType"])

In [ ]:
df["fuelType"].value_counts()

In [ ]:
filt=(df["fuelType"]=="Other")
df.drop(df.loc[filt].index,inplace=True)

In [ ]:
del df["conv_fuelType"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbe=LabelEncoder()
df["conv_fuelType"]=lbe.fit_transform(df["fuelType"])

In [ ]:
del df["fuelType"]

In [ ]:
df=pd.get_dummies(df,columns=["transmission"],prefix=["transmission"])

In [ ]:
filt=((df["model"]==" GTC") | (df["model"]==" Combo Life") | (df["model"]==" Meriva"))
df.drop(df.loc[filt].index,inplace=True)

#df2=pd.get_dummies(df,columns=["model"],prefix=["model"])

In [ ]:
df["model"].value_counts()

In [ ]:
df2=pd.get_dummies(df,columns=["model"],prefix=["model"])

In [ ]:
df2

In [ ]:
y=df2["price"]
X=df2.drop("price",axis=1)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
gbm_model=GradientBoostingRegressor().fit(X_train,y_train)
y_pred=gbm_model.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_pred,y_test))
print("rmse:",rmse)

In [ ]:
model=Ridge().fit(X_train,y_train)

y_pred=model.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_pred,y_test))
print("rmse:",rmse)

In [ ]:
gbm_params={"max_depth":[2],"n_estimators":[500,1000],"min_samples_split":[30,40]}
gbm_model=GradientBoostingRegressor()
gbm_cv=GridSearchCV(gbm_model,gbm_params,cv=5,verbose=2).fit(X_train,y_train)

In [ ]:
gbm_cv.best_params_

In [ ]:
gbm_tuned=GradientBoostingRegressor(max_depth=2,min_samples_split=40,n_estimators=500).fit(X_train,y_train)

In [ ]:
y_pred=gbm_tuned.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_pred,y_test))
print(rmse)

In [ ]:
sns.boxplot(data=df,x="price");

In [ ]:
Importance=pd.DataFrame({"Importance":gbm_tuned.feature_importances_*100},index=X_train.columns)

In [ ]:
Importance.sort_values(by="Importance",axis=0,ascending=True).plot(kind="barh",color="r")

In [ ]:
dfy=df.drop(["conv_fuelType","transmission_Automatic","transmission_Semi-Auto","engineSize"],axis=1)

In [ ]:
dfy

In [ ]:
dfyy=pd.get_dummies(dfy,columns=["model"],prefix=["model"])

In [ ]:
dfyy.head()

In [ ]:
dfyy.drop(["model_ Adam","model_ Astra","model_ Crossland X","model_ Mokka","model_ Mokka X","model_ Zafira"],axis=1,inplace=True)

In [ ]:
y=dfyy["price"]
X=dfyy.drop("price",axis=1)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
gbm_model=GradientBoostingRegressor().fit(X_train,y_train)
y_pred=gbm_model.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_pred,y_test))
print("rmse:",rmse)

In [ ]:
gbm_tuned=GradientBoostingRegressor(max_depth=2,min_samples_split=40,n_estimators=500).fit(X_train,y_train)
y_pred=gbm_tuned.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_pred,y_test))
print(rmse)

In [ ]:
Importance=pd.DataFrame({"Importance":gbm_tuned.feature_importances_*100},index=X_train.columns)
Importance.sort_values(by="Importance",axis=0,ascending=True).plot(kind="barh",color="r")